In [2]:
import sys
sys.path.append('../')
from resnet_quant import *
from resnet import *

In [3]:
# Importing model
qparams = [4, 4, 'dorefa'] # [abit, qbit, quant_method] --- only dorefa has been implemented

# resnet models are 8, 14, 16, 24

# activation and weight quantization are computed slightly different.
quant_resnet = get_quant_model('resnet8', qparams)


In [4]:
# Note: first and last layer are not quantized 
# looking at layer ouptuts using hooks
outputs=[]
quant_resnet = get_quant_model('resnet8', qparams)


def hook(module, input, output):
    outputs.append(output)

# Registering quantized act and 
for n, m in quant_resnet.named_modules():
    if n == 'conv1': # first layer output
        m.register_forward_hook(hook)
    elif 'qfn' in n:
        m.register_forward_hook(hook)
    elif 'fc' in n:
        m.register_forward_hook(hook)
import torch
# reset model or it will continue appending outputs 
image = torch.unsqueeze(torch.rand(size=(3, 32, 32)), dim=0)
out = quant_resnet(image)


In [5]:
# look at some values at intermediate layer outputs
# weights and acitvations are quantized but operations are still in full precision
# with the final layer also full precision

for output in outputs:
    output = output.view(1, -1)
    print(np.squeeze(output.data.numpy()[0:5]))

[ 0.12953429  0.12341644  0.26982716 ...  0.07344282  0.09889573
 -0.02507184]
[0.6666667 0.6       1.        ... 0.        0.        0.       ]
[ 0.07930576  0.07930576 -0.13217624 ... -0.07930574  0.07930576
 -0.13217624]
[0.  0.  0.  ... 0.2 1.  1. ]
[-0.175937    0.1256693  -0.27647245 ... -0.07540157  0.1256693
  0.07540159]
[0.6666667 1.        0.        ... 0.        0.        0.       ]
[-0.02174077 -0.06522232 -0.02174077 ...  0.15218543 -0.10870387
 -0.15218541]
[0.         0.53333336 0.         ... 1.         0.6666667  1.        ]
[-0.02142314  0.02142316 -0.06426942 ... -0.02142314 -0.02142314
  0.06426943]
[1.  0.4 1.  ... 1.  0.  0.8]
[ 0.07327925  0.07327925  0.07327925 ... -0.04396754 -0.10259093
 -0.01465585]
[0.46666667 1.         0.33333334 ... 0.         0.         0.93333334]
[ 0.07887743 -0.04732645 -0.04732645 ... -0.07887741  0.04732646
  0.0157755 ]
[ 0.13122779  0.07147981  0.14192243  0.07814884 -0.39279425 -0.09664904
 -0.54337305 -0.01641652  0.08411925 -0

In [6]:
# print outputs
# simple loss function to look at gradients
loss = torch.sum(out)
print(out.grad)

None


In [7]:
resnet = get_model('resnet8')
torch.quantization.prepare_qat(resnet, inplace=True)
print(resnet.features[1].conv)

/Users/btl787/anaconda3/lib/python3.7/site-packages/torch/quantization/quantize.py:147: UserWarning: None of the submodule got qconfig applied. Make sure you passed correct configuration through `qconfig_dict` or by assigning the `.qconfig` attribute directly on submodules
  warnings.warn("None of the submodule got qconfig applied. Make sure you "


AttributeError: 'ResNet_Cifar' object has no attribute 'features'